In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
'''
FEATURES:
Overall action count/ratio      DONE
Overall day count               DONE
Monthly action count/ratio  TODO
Product Diversity               DONE
Monthly aggregation         TODO
Merchant aggregation        TODO
Double 11 features              DONE
latest one-week                 DONE
trend                       TODO
LDA features                TODO
'''

'\nFEATURES:\nOverall action count/ratio\nOverall day count\nMonthly action count/ratio\nProduct Diversity\nMonthly aggregation\nMerchant aggregation\nDouble 11 features\nlatest one-week\ntrend\nLDA features\n'

In [3]:
df = pd.read_csv("./use_data/expanded_training.csv")
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0


In [4]:
df_user = df[['user_id']]
df_user

,user_id
0,379824
1,379824
2,379824
3,379824
4,379824
...,...
380,122632
381,122632
382,122632
383,95362


In [5]:
#FEATURE 1: OVERALL ACTION COUNT/RATIO
actions = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['user_id'] not in actions:
        # there may be multiple entries, so default with an empty list
        actions[row["user_id"]] = [] 
    actions[row["user_id"]].append(row["action_type"])

# DANIEL: taking out unique value lists    
#for k,v in actions.items():
#    actions[k] = list(set(v)) # make each value list unique
actions

{379824.0: [0.0, 0.0, 2.0, 0.0, 0.0, 0.0],
 141307.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 252255.0: [0.0],
 309566.0: [2.0],
 206032.0: [0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0],
 182119.0: [0.0, 0.0],
 68225.0: [0.0],
 109919.0: [0.0],
 342023.0: [0.0, 0.0],
 287829.0: [0.0],
 255960.0: [0.0],
 380420.0: [0.0],
 176150.0: [0.0, 0.0],
 407671.0: [0.0, 2.0, 0.0, 3.0, 0.0, 0.0],
 388332.0: [0.0],
 244552.0: [2.0, 0.0],
 46507.0: [0.0, 0.0, 2.0],
 191162.0: [0.0, 2.0],
 174308.0: [0.0, 0.0, 2.0, 0.0],
 134532.0: [0.0],
 355159.0: [0.0],
 62942.0: [0.0],
 353962.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 319670.0: [0.0, 0.0, 2.0],
 210578.0: [0.0, 0.0, 0.0],
 101110.0: [2.0, 0.0, 0.0, 0.0],
 149798.0: [0.0],
 6018.0: [0.0, 0.0],
 281262.0: [0.0],
 314519.0: [2.0],
 205257.0: [0.0],
 117839.0: [0.0],
 46699.0: [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0],
 238412.0: [0.0, 0.0, 0.0, 0.0],
 156961.0: [0.0],
 4557

In [5]:
#for k,v in actions.items():
#    actions[k] = list(set(v)) # make each value list unique

In [6]:
actions

{379824.0: [0.0, 0.0, 2.0, 0.0, 0.0, 0.0],
 141307.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 252255.0: [0.0],
 309566.0: [2.0],
 206032.0: [0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0],
 182119.0: [0.0, 0.0],
 68225.0: [0.0],
 109919.0: [0.0],
 342023.0: [0.0, 0.0],
 287829.0: [0.0],
 255960.0: [0.0],
 380420.0: [0.0],
 176150.0: [0.0, 0.0],
 407671.0: [0.0, 2.0, 0.0, 3.0, 0.0, 0.0],
 388332.0: [0.0],
 244552.0: [2.0, 0.0],
 46507.0: [0.0, 0.0, 2.0],
 191162.0: [0.0, 2.0],
 174308.0: [0.0, 0.0, 2.0, 0.0],
 134532.0: [0.0],
 355159.0: [0.0],
 62942.0: [0.0],
 353962.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 319670.0: [0.0, 0.0, 2.0],
 210578.0: [0.0, 0.0, 0.0],
 101110.0: [2.0, 0.0, 0.0, 0.0],
 149798.0: [0.0],
 6018.0: [0.0, 0.0],
 281262.0: [0.0],
 314519.0: [2.0],
 205257.0: [0.0],
 117839.0: [0.0],
 46699.0: [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0],
 238412.0: [0.0, 0.0, 0.0, 0.0],
 156961.0: [0.0],
 4557

In [7]:
action_count = {}
for key in actions.keys():
    action_count[key] = len(actions[key])
action_count

{379824.0: 6,
 141307.0: 9,
 252255.0: 1,
 309566.0: 1,
 206032.0: 14,
 182119.0: 2,
 68225.0: 1,
 109919.0: 1,
 342023.0: 2,
 287829.0: 1,
 255960.0: 1,
 380420.0: 1,
 176150.0: 2,
 407671.0: 6,
 388332.0: 1,
 244552.0: 2,
 46507.0: 3,
 191162.0: 2,
 174308.0: 4,
 134532.0: 1,
 355159.0: 1,
 62942.0: 1,
 353962.0: 6,
 319670.0: 3,
 210578.0: 3,
 101110.0: 4,
 149798.0: 1,
 6018.0: 2,
 281262.0: 1,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 13,
 238412.0: 4,
 156961.0: 1,
 45576.0: 6,
 214766.0: 2,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 402251.0: 4,
 254609.0: 4,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 291386.0: 1,
 313634.0: 4,
 60260.0: 1,
 112997.0: 2,
 305925.0: 2,
 199429.0: 1,
 185381.0: 1,
 144281.0: 2,
 33618.0: 1,
 276857.0: 3,
 130186.0: 4,
 376338.0: 4,
 867.0: 1,
 338395.0: 2,
 335587.0: 6,
 409283.0: 1,
 88275.0: 1,
 165355.0: 5,
 55784.0: 1,
 327782.0: 8,
 264657.0: 6,
 52751.0: 2,
 33745.0: 4,
 103980.0: 1,
 316235.0

In [8]:
total_action_count = sum(action_count.values())
total_action_count

385

In [9]:
action_count_ratio = {}
for key in action_count:
    action_count_ratio[key] = action_count[key]/total_action_count
action_count_ratio

{379824.0: 0.015584415584415584,
 141307.0: 0.023376623376623377,
 252255.0: 0.0025974025974025974,
 309566.0: 0.0025974025974025974,
 206032.0: 0.03636363636363636,
 182119.0: 0.005194805194805195,
 68225.0: 0.0025974025974025974,
 109919.0: 0.0025974025974025974,
 342023.0: 0.005194805194805195,
 287829.0: 0.0025974025974025974,
 255960.0: 0.0025974025974025974,
 380420.0: 0.0025974025974025974,
 176150.0: 0.005194805194805195,
 407671.0: 0.015584415584415584,
 388332.0: 0.0025974025974025974,
 244552.0: 0.005194805194805195,
 46507.0: 0.007792207792207792,
 191162.0: 0.005194805194805195,
 174308.0: 0.01038961038961039,
 134532.0: 0.0025974025974025974,
 355159.0: 0.0025974025974025974,
 62942.0: 0.0025974025974025974,
 353962.0: 0.015584415584415584,
 319670.0: 0.007792207792207792,
 210578.0: 0.007792207792207792,
 101110.0: 0.01038961038961039,
 149798.0: 0.0025974025974025974,
 6018.0: 0.005194805194805195,
 281262.0: 0.0025974025974025974,
 314519.0: 0.0025974025974025974,
 205

In [10]:
df_user['action count'] = -1
df_user['action count ratio'] = -1
for index,row in df_user.iterrows():
        df_user.at[index, 'action count'] = action_count[row['user_id']]
        df_user.at[index, 'action count ratio'] = action_count_ratio[row['user_id']]

df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\873788209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['action count'] = -1
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\873788209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['action count ratio'] = -1


,user_id,action count,action count ratio
0,379824,6,0.015584
1,379824,6,0.015584
2,379824,6,0.015584
3,379824,6,0.015584
4,379824,6,0.015584
...,...,...,...
380,122632,3,0.007792
381,122632,3,0.007792
382,122632,3,0.007792
383,95362,1,0.002597


In [11]:
#FEATURE 2: OVERALL DAY COUNT
days = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['user_id'] not in days:
        # there may be multiple entries, so default with an empty list
        days[row["user_id"]] = [] 
    days[row["user_id"]].append(row["time_stamp"])

In [13]:
for k,v in days.items():
    days[k] = list(set(v)) # make each value list unique

In [14]:
days

{379824.0: [1110.0, 1111.0],
 141307.0: [1107.0, 1111.0],
 252255.0: [1111.0],
 309566.0: [1111.0],
 206032.0: [1103.0, 1104.0, 1105.0, 1107.0, 1108.0, 1109.0, 1111.0],
 182119.0: [1110.0, 1111.0],
 68225.0: [1108.0],
 109919.0: [1105.0],
 342023.0: [1108.0],
 287829.0: [1111.0],
 255960.0: [1111.0],
 380420.0: [1105.0],
 176150.0: [1111.0],
 407671.0: [1109.0, 1111.0],
 388332.0: [520.0],
 244552.0: [1111.0],
 46507.0: [1111.0],
 191162.0: [1111.0],
 174308.0: [1111.0],
 134532.0: [1111.0],
 355159.0: [1111.0],
 62942.0: [1103.0],
 353962.0: [1111.0],
 319670.0: [1111.0],
 210578.0: [1111.0],
 101110.0: [1111.0],
 149798.0: [1108.0],
 6018.0: [1111.0],
 281262.0: [1109.0],
 314519.0: [1111.0],
 205257.0: [1111.0],
 117839.0: [1111.0],
 46699.0: [1106.0, 1109.0, 1110.0, 1111.0],
 238412.0: [1111.0],
 156961.0: [1111.0],
 45576.0: [1111.0],
 214766.0: [1110.0],
 5524.0: [1111.0],
 218669.0: [1111.0],
 272518.0: [1111.0],
 277770.0: [1111.0],
 386295.0: [1111.0],
 402251.0: [1109.0, 1103

In [15]:
day_count = {}
for key in days.keys():
    day_count[key] = len(days[key])
day_count

{379824.0: 2,
 141307.0: 2,
 252255.0: 1,
 309566.0: 1,
 206032.0: 7,
 182119.0: 2,
 68225.0: 1,
 109919.0: 1,
 342023.0: 1,
 287829.0: 1,
 255960.0: 1,
 380420.0: 1,
 176150.0: 1,
 407671.0: 2,
 388332.0: 1,
 244552.0: 1,
 46507.0: 1,
 191162.0: 1,
 174308.0: 1,
 134532.0: 1,
 355159.0: 1,
 62942.0: 1,
 353962.0: 1,
 319670.0: 1,
 210578.0: 1,
 101110.0: 1,
 149798.0: 1,
 6018.0: 1,
 281262.0: 1,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 4,
 238412.0: 1,
 156961.0: 1,
 45576.0: 1,
 214766.0: 1,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 402251.0: 2,
 254609.0: 2,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 291386.0: 1,
 313634.0: 1,
 60260.0: 1,
 112997.0: 1,
 305925.0: 1,
 199429.0: 1,
 185381.0: 1,
 144281.0: 1,
 33618.0: 1,
 276857.0: 1,
 130186.0: 2,
 376338.0: 1,
 867.0: 1,
 338395.0: 2,
 335587.0: 1,
 409283.0: 1,
 88275.0: 1,
 165355.0: 3,
 55784.0: 1,
 327782.0: 4,
 264657.0: 1,
 52751.0: 1,
 33745.0: 1,
 103980.0: 1,
 316235.0: 

In [16]:
df_user['day count'] = -1
for index,row in df_user.iterrows():
        df_user.at[index, 'day count'] = day_count[row['user_id']]

df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\2569712461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['day count'] = -1


,user_id,action count,action count ratio,day count
0,379824,6,0.015584,2
1,379824,6,0.015584,2
2,379824,6,0.015584,2
3,379824,6,0.015584,2
4,379824,6,0.015584,2
...,...,...,...,...
380,122632,3,0.007792,2
381,122632,3,0.007792,2
382,122632,3,0.007792,2
383,95362,1,0.002597,1


In [17]:
#FEATURE 4: Product Diversity
items = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['item_id'] not in items:
        # there may be multiple entries, so default with an empty list
        items[row["user_id"]] = [] 
    items[row["user_id"]].append(row["item_id"])

In [19]:
for k,v in items.items():
    items[k] = list(set(v)) # make each value list unique

In [18]:
items

{379824.0: [198.0],
 141307.0: [175.0],
 252255.0: [175.0],
 309566.0: [175.0],
 206032.0: [281.0],
 182119.0: [279.0],
 68225.0: [279.0],
 109919.0: [198.0],
 342023.0: [190.0],
 287829.0: [175.0],
 255960.0: [279.0],
 380420.0: [190.0],
 176150.0: [175.0],
 407671.0: [219.0],
 388332.0: [285.0],
 244552.0: [278.0],
 46507.0: [224.0],
 191162.0: [184.0],
 174308.0: [175.0],
 134532.0: [279.0],
 355159.0: [279.0],
 62942.0: [312.0],
 353962.0: [184.0],
 319670.0: [175.0],
 210578.0: [279.0],
 101110.0: [281.0],
 149798.0: [224.0],
 6018.0: [184.0],
 281262.0: [279.0],
 314519.0: [219.0],
 205257.0: [279.0],
 117839.0: [279.0],
 46699.0: [175.0],
 238412.0: [253.0],
 156961.0: [279.0],
 45576.0: [279.0],
 214766.0: [253.0],
 5524.0: [224.0],
 218669.0: [279.0],
 272518.0: [224.0],
 277770.0: [279.0],
 386295.0: [279.0],
 402251.0: [175.0],
 254609.0: [279.0],
 84043.0: [175.0],
 120820.0: [175.0],
 291453.0: [274.0],
 280293.0: [184.0],
 291386.0: [279.0],
 313634.0: [226.0],
 60260.0: 

In [20]:
item_count = {}
for key in items.keys():
    item_count[key] = len(items[key])
item_count

{379824.0: 1,
 141307.0: 1,
 252255.0: 1,
 309566.0: 1,
 206032.0: 1,
 182119.0: 1,
 68225.0: 1,
 109919.0: 1,
 342023.0: 1,
 287829.0: 1,
 255960.0: 1,
 380420.0: 1,
 176150.0: 1,
 407671.0: 1,
 388332.0: 1,
 244552.0: 1,
 46507.0: 1,
 191162.0: 1,
 174308.0: 1,
 134532.0: 1,
 355159.0: 1,
 62942.0: 1,
 353962.0: 1,
 319670.0: 1,
 210578.0: 1,
 101110.0: 1,
 149798.0: 1,
 6018.0: 1,
 281262.0: 1,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 1,
 238412.0: 1,
 156961.0: 1,
 45576.0: 1,
 214766.0: 1,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 402251.0: 1,
 254609.0: 1,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 291386.0: 1,
 313634.0: 1,
 60260.0: 1,
 112997.0: 1,
 305925.0: 1,
 199429.0: 1,
 185381.0: 1,
 144281.0: 1,
 33618.0: 1,
 276857.0: 1,
 130186.0: 1,
 376338.0: 1,
 867.0: 1,
 338395.0: 1,
 335587.0: 1,
 409283.0: 1,
 88275.0: 1,
 165355.0: 1,
 55784.0: 1,
 327782.0: 1,
 264657.0: 1,
 52751.0: 1,
 33745.0: 1,
 103980.0: 1,
 316235.0: 

In [21]:
total_item_count = sum(item_count.values())
total_item_count

169

In [22]:
item_count_diversity = {}
for key in item_count:
    item_count_diversity[key]= item_count[key]/total_item_count
item_count_diversity

{379824.0: 0.005917159763313609,
 141307.0: 0.005917159763313609,
 252255.0: 0.005917159763313609,
 309566.0: 0.005917159763313609,
 206032.0: 0.005917159763313609,
 182119.0: 0.005917159763313609,
 68225.0: 0.005917159763313609,
 109919.0: 0.005917159763313609,
 342023.0: 0.005917159763313609,
 287829.0: 0.005917159763313609,
 255960.0: 0.005917159763313609,
 380420.0: 0.005917159763313609,
 176150.0: 0.005917159763313609,
 407671.0: 0.005917159763313609,
 388332.0: 0.005917159763313609,
 244552.0: 0.005917159763313609,
 46507.0: 0.005917159763313609,
 191162.0: 0.005917159763313609,
 174308.0: 0.005917159763313609,
 134532.0: 0.005917159763313609,
 355159.0: 0.005917159763313609,
 62942.0: 0.005917159763313609,
 353962.0: 0.005917159763313609,
 319670.0: 0.005917159763313609,
 210578.0: 0.005917159763313609,
 101110.0: 0.005917159763313609,
 149798.0: 0.005917159763313609,
 6018.0: 0.005917159763313609,
 281262.0: 0.005917159763313609,
 314519.0: 0.005917159763313609,
 205257.0: 0.00

In [23]:
df_user['product diversity'] = -1
for index,row in df_user.iterrows():
        df_user.at[index, 'product diversity'] = item_count_diversity[row['user_id']]

df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\3681883862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['product diversity'] = -1


,user_id,action count,action count ratio,day count,product diversity
0,379824,6,0.015584,2,0.005917
1,379824,6,0.015584,2,0.005917
2,379824,6,0.015584,2,0.005917
3,379824,6,0.015584,2,0.005917
4,379824,6,0.015584,2,0.005917
...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917
381,122632,3,0.007792,2,0.005917
382,122632,3,0.007792,2,0.005917
383,95362,1,0.002597,1,0.005917


In [23]:
#FEATURE 9: DOUBLE 11 FEATURES

In [24]:
df_11 = df[df['time_stamp']==1111]
df_11

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
9,141307,175,1181,4760,247.0,1111,0,4.0,1.0,0
10,141307,175,1181,4760,247.0,1111,0,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
375,289079,279,898,3323,683.0,1111,0,4.0,1.0,1
377,403117,175,1181,4760,247.0,1111,0,2.0,1.0,0
378,36385,219,349,1943,6208.0,1111,2,0.0,0.0,0
379,36385,219,349,1943,6208.0,1111,0,0.0,0.0,0


In [25]:
actions_11 = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df_11.iterrows():
    if row['user_id'] not in actions_11:
        # there may be multiple entries, so default with an empty list
        actions_11[row["user_id"]] = [] 
    actions_11[row["user_id"]].append(row["action_type"])

In [26]:
#for k,v in actions_11.items():
#    actions_11[k] = list(set(v)) # make each value list unique
actions_11

{379824.0: [0.0, 0.0, 2.0],
 141307.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 252255.0: [0.0],
 309566.0: [2.0],
 206032.0: [2.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 182119.0: [0.0],
 287829.0: [0.0],
 255960.0: [0.0],
 176150.0: [0.0, 0.0],
 407671.0: [0.0, 2.0, 0.0],
 244552.0: [2.0, 0.0],
 46507.0: [0.0, 0.0, 2.0],
 191162.0: [0.0, 2.0],
 174308.0: [0.0, 0.0, 2.0, 0.0],
 134532.0: [0.0],
 355159.0: [0.0],
 353962.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 319670.0: [0.0, 0.0, 2.0],
 210578.0: [0.0, 0.0, 0.0],
 101110.0: [2.0, 0.0, 0.0, 0.0],
 6018.0: [0.0, 0.0],
 314519.0: [2.0],
 205257.0: [0.0],
 117839.0: [0.0],
 46699.0: [2.0],
 238412.0: [0.0, 0.0, 0.0, 0.0],
 156961.0: [0.0],
 45576.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 5524.0: [0.0],
 218669.0: [0.0],
 272518.0: [0.0],
 277770.0: [0.0],
 386295.0: [0.0],
 254609.0: [0.0, 0.0],
 84043.0: [0.0],
 120820.0: [0.0],
 291453.0: [0.0],
 280293.0: [0.0],
 313634.0: [0.0, 0.0, 0.0, 2.0],
 60260.0: [0.0],
 305925.0: [0.0, 3.0],
 185381.0: [0.0],
 144281.0: [0

In [27]:
action_count_11 = {}
for key in actions_11.keys():
    action_count_11[key] = len(actions_11[key])
action_count_11

{379824.0: 3,
 141307.0: 6,
 252255.0: 1,
 309566.0: 1,
 206032.0: 6,
 182119.0: 1,
 287829.0: 1,
 255960.0: 1,
 176150.0: 2,
 407671.0: 3,
 244552.0: 2,
 46507.0: 3,
 191162.0: 2,
 174308.0: 4,
 134532.0: 1,
 355159.0: 1,
 353962.0: 6,
 319670.0: 3,
 210578.0: 3,
 101110.0: 4,
 6018.0: 2,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 1,
 238412.0: 4,
 156961.0: 1,
 45576.0: 6,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 254609.0: 2,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 313634.0: 4,
 60260.0: 1,
 305925.0: 2,
 185381.0: 1,
 144281.0: 2,
 33618.0: 1,
 276857.0: 3,
 130186.0: 1,
 376338.0: 4,
 867.0: 1,
 338395.0: 1,
 335587.0: 6,
 409283.0: 1,
 165355.0: 2,
 55784.0: 1,
 327782.0: 1,
 264657.0: 6,
 52751.0: 2,
 33745.0: 4,
 316235.0: 7,
 380439.0: 1,
 408759.0: 2,
 38606.0: 2,
 108672.0: 1,
 275260.0: 6,
 53531.0: 1,
 51728.0: 6,
 177770.0: 1,
 389062.0: 3,
 383379.0: 1,
 165262.0: 2,
 97492.0: 1,
 335815.0: 1,
 8552.0: 1,
 340708.0: 3,


In [28]:
total_action_count_11 = sum(action_count_11.values())
total_action_count_11

228

In [29]:
action_count_ratio_11 = {}
for key in action_count_11:
    action_count_ratio_11[key] = action_count_11[key]/total_action_count_11
action_count_ratio_11

{379824.0: 0.013157894736842105,
 141307.0: 0.02631578947368421,
 252255.0: 0.0043859649122807015,
 309566.0: 0.0043859649122807015,
 206032.0: 0.02631578947368421,
 182119.0: 0.0043859649122807015,
 287829.0: 0.0043859649122807015,
 255960.0: 0.0043859649122807015,
 176150.0: 0.008771929824561403,
 407671.0: 0.013157894736842105,
 244552.0: 0.008771929824561403,
 46507.0: 0.013157894736842105,
 191162.0: 0.008771929824561403,
 174308.0: 0.017543859649122806,
 134532.0: 0.0043859649122807015,
 355159.0: 0.0043859649122807015,
 353962.0: 0.02631578947368421,
 319670.0: 0.013157894736842105,
 210578.0: 0.013157894736842105,
 101110.0: 0.017543859649122806,
 6018.0: 0.008771929824561403,
 314519.0: 0.0043859649122807015,
 205257.0: 0.0043859649122807015,
 117839.0: 0.0043859649122807015,
 46699.0: 0.0043859649122807015,
 238412.0: 0.017543859649122806,
 156961.0: 0.0043859649122807015,
 45576.0: 0.02631578947368421,
 5524.0: 0.0043859649122807015,
 218669.0: 0.0043859649122807015,
 272518

In [30]:
df_user['1111 action count'] = 0
df_user['1111 action count ratio'] = 0
for index,row in df_user.iterrows():
        if row['user_id'] in action_count_11.keys():
            df_user.at[index, '1111 action count'] = action_count_11[row['user_id']]
        if row['user_id'] in action_count_ratio_11.keys():
            df_user.at[index, '1111 action count ratio'] = action_count_ratio_11[row['user_id']]

df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\290636859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['1111 action count'] = 0
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\290636859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['1111 action count ratio'] = 0


,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio
0,379824,6,0.015584,2,0.005917,3,0.013158
1,379824,6,0.015584,2,0.005917,3,0.013158
2,379824,6,0.015584,2,0.005917,3,0.013158
3,379824,6,0.015584,2,0.005917,3,0.013158
4,379824,6,0.015584,2,0.005917,3,0.013158
...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000
381,122632,3,0.007792,2,0.005917,0,0.000000
382,122632,3,0.007792,2,0.005917,0,0.000000
383,95362,1,0.002597,1,0.005917,1,0.004386


In [31]:
activity_ratio_1111 = {}
df_user['1111 activity ratio'] = 0
for index,row in df_user.iterrows():
    if df_user.at[index,'1111 action count'] != 0:
        df_user.at[index, '1111 activity ratio'] = (df_user.at[index,'1111 action count']/df_user.at[index,'action count'])
df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\1757874129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['1111 activity ratio'] = 0


,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio,1111 activity ratio
0,379824,6,0.015584,2,0.005917,3,0.013158,0.5
1,379824,6,0.015584,2,0.005917,3,0.013158,0.5
2,379824,6,0.015584,2,0.005917,3,0.013158,0.5
3,379824,6,0.015584,2,0.005917,3,0.013158,0.5
4,379824,6,0.015584,2,0.005917,3,0.013158,0.5
...,...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000,0.0
381,122632,3,0.007792,2,0.005917,0,0.000000,0.0
382,122632,3,0.007792,2,0.005917,0,0.000000,0.0
383,95362,1,0.002597,1,0.005917,1,0.004386,1.0


In [32]:
#FEATURE 10: LATEST ONE-WEEK
latest_week = 1104
df_latest_week = df[df['time_stamp']>=latest_week]
df_latest_week

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0


In [33]:
actions_last_week = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df_latest_week.iterrows():
    if row['user_id'] not in actions_last_week:
        # there may be multiple entries, so default with an empty list
        actions_last_week[row["user_id"]] = [] 
    actions_last_week[row["user_id"]].append(row["action_type"])
    
#for k,v in actions_last_week.items():
#    actions_last_week[k] = list(set(v)) # make each value list unique
actions_last_week

{379824.0: [0.0, 0.0, 2.0, 0.0, 0.0, 0.0],
 141307.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 252255.0: [0.0],
 309566.0: [2.0],
 206032.0: [0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0],
 182119.0: [0.0, 0.0],
 68225.0: [0.0],
 109919.0: [0.0],
 342023.0: [0.0, 0.0],
 287829.0: [0.0],
 255960.0: [0.0],
 380420.0: [0.0],
 176150.0: [0.0, 0.0],
 407671.0: [0.0, 2.0, 0.0, 3.0, 0.0, 0.0],
 244552.0: [2.0, 0.0],
 46507.0: [0.0, 0.0, 2.0],
 191162.0: [0.0, 2.0],
 174308.0: [0.0, 0.0, 2.0, 0.0],
 134532.0: [0.0],
 355159.0: [0.0],
 353962.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 319670.0: [0.0, 0.0, 2.0],
 210578.0: [0.0, 0.0, 0.0],
 101110.0: [2.0, 0.0, 0.0, 0.0],
 149798.0: [0.0],
 6018.0: [0.0, 0.0],
 281262.0: [0.0],
 314519.0: [2.0],
 205257.0: [0.0],
 117839.0: [0.0],
 46699.0: [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0],
 238412.0: [0.0, 0.0, 0.0, 0.0],
 156961.0: [0.0],
 45576.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 214766.0: [0.0, 0.0],
 5524.

In [34]:
action_count_last_week = {}
for key in actions_last_week.keys():
    action_count_last_week[key] = len(actions_last_week[key])
action_count_last_week

{379824.0: 6,
 141307.0: 9,
 252255.0: 1,
 309566.0: 1,
 206032.0: 13,
 182119.0: 2,
 68225.0: 1,
 109919.0: 1,
 342023.0: 2,
 287829.0: 1,
 255960.0: 1,
 380420.0: 1,
 176150.0: 2,
 407671.0: 6,
 244552.0: 2,
 46507.0: 3,
 191162.0: 2,
 174308.0: 4,
 134532.0: 1,
 355159.0: 1,
 353962.0: 6,
 319670.0: 3,
 210578.0: 3,
 101110.0: 4,
 149798.0: 1,
 6018.0: 2,
 281262.0: 1,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 13,
 238412.0: 4,
 156961.0: 1,
 45576.0: 6,
 214766.0: 2,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 402251.0: 2,
 254609.0: 4,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 291386.0: 1,
 313634.0: 4,
 60260.0: 1,
 112997.0: 2,
 305925.0: 2,
 199429.0: 1,
 185381.0: 1,
 144281.0: 2,
 33618.0: 1,
 276857.0: 3,
 130186.0: 4,
 376338.0: 4,
 867.0: 1,
 338395.0: 2,
 335587.0: 6,
 409283.0: 1,
 165355.0: 5,
 55784.0: 1,
 327782.0: 8,
 264657.0: 6,
 52751.0: 2,
 33745.0: 4,
 316235.0: 10,
 380439.0: 2,
 86171.0: 1,
 408759.0: 2,
 38606.

In [35]:
total_action_count_last_week = sum(action_count_last_week.values())
total_action_count_last_week

355

In [36]:
action_count_ratio_last_week = {}
for key in action_count_last_week:
    action_count_ratio_last_week[key] = action_count_last_week[key]/total_action_count_last_week
action_count_ratio_last_week

{379824.0: 0.016901408450704224,
 141307.0: 0.02535211267605634,
 252255.0: 0.0028169014084507044,
 309566.0: 0.0028169014084507044,
 206032.0: 0.036619718309859155,
 182119.0: 0.005633802816901409,
 68225.0: 0.0028169014084507044,
 109919.0: 0.0028169014084507044,
 342023.0: 0.005633802816901409,
 287829.0: 0.0028169014084507044,
 255960.0: 0.0028169014084507044,
 380420.0: 0.0028169014084507044,
 176150.0: 0.005633802816901409,
 407671.0: 0.016901408450704224,
 244552.0: 0.005633802816901409,
 46507.0: 0.008450704225352112,
 191162.0: 0.005633802816901409,
 174308.0: 0.011267605633802818,
 134532.0: 0.0028169014084507044,
 355159.0: 0.0028169014084507044,
 353962.0: 0.016901408450704224,
 319670.0: 0.008450704225352112,
 210578.0: 0.008450704225352112,
 101110.0: 0.011267605633802818,
 149798.0: 0.0028169014084507044,
 6018.0: 0.005633802816901409,
 281262.0: 0.0028169014084507044,
 314519.0: 0.0028169014084507044,
 205257.0: 0.0028169014084507044,
 117839.0: 0.0028169014084507044,
 

In [37]:
df_user['last week action count'] = 0
df_user['last week action count ratio'] = 0
for index,row in df_user.iterrows():
        if row['user_id'] in action_count_last_week.keys():
            df_user.at[index, 'last week action count'] = action_count_last_week[row['user_id']]
        if row['user_id'] in action_count_ratio_last_week.keys():
            df_user.at[index, 'last week action count ratio'] = action_count_ratio_last_week[row['user_id']]

df_user

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16936\602709202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['last week action count'] = 0


,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio,1111 activity ratio,last week action count,last week action count ratio
0,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901
1,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901
2,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901
3,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901
4,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901
...,...,...,...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451
381,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451
382,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451
383,95362,1,0.002597,1,0.005917,1,0.004386,1.0,1,0.002817


In [38]:
activity_ratio_last_week = {}
df_user['last week activity ratio'] = 0
for index,row in df_user.iterrows():
    if df_user.at[index,'last week action count'] != 0:
        df_user.at[index, 'last week activity ratio'] = (df_user.at[index,'last week action count']/df_user.at[index,'action count'])
df_user

,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio,1111 activity ratio,last week action count,last week action count ratio,last week activity ratio
0,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0
1,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0
2,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0
3,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0
4,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0
...,...,...,...,...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0
381,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0
382,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0
383,95362,1,0.002597,1,0.005917,1,0.004386,1.0,1,0.002817,1.0


In [39]:
#FEATURE 11: LATEST MONTH
latest_month = 1011
df_latest_month = df[df['time_stamp']>=latest_month]
df_latest_month

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0


In [41]:
actions_last_month = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df_latest_month.iterrows():
    if row['user_id'] not in actions_last_month:
        # there may be multiple entries, so default with an empty list
        actions_last_month[row["user_id"]] = [] 
    actions_last_month[row["user_id"]].append(row["action_type"])
    
#for k,v in actions_last_month.items():
#    actions_last_month[k] = list(set(v)) # make each value list unique
actions_last_month

{379824.0: [0.0, 0.0, 2.0, 0.0, 0.0, 0.0],
 141307.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 252255.0: [0.0],
 309566.0: [2.0],
 206032.0: [0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0],
 182119.0: [0.0, 0.0],
 68225.0: [0.0],
 109919.0: [0.0],
 342023.0: [0.0, 0.0],
 287829.0: [0.0],
 255960.0: [0.0],
 380420.0: [0.0],
 176150.0: [0.0, 0.0],
 407671.0: [0.0, 2.0, 0.0, 3.0, 0.0, 0.0],
 244552.0: [2.0, 0.0],
 46507.0: [0.0, 0.0, 2.0],
 191162.0: [0.0, 2.0],
 174308.0: [0.0, 0.0, 2.0, 0.0],
 134532.0: [0.0],
 355159.0: [0.0],
 62942.0: [0.0],
 353962.0: [0.0, 0.0, 0.0, 2.0, 0.0, 0.0],
 319670.0: [0.0, 0.0, 2.0],
 210578.0: [0.0, 0.0, 0.0],
 101110.0: [2.0, 0.0, 0.0, 0.0],
 149798.0: [0.0],
 6018.0: [0.0, 0.0],
 281262.0: [0.0],
 314519.0: [2.0],
 205257.0: [0.0],
 117839.0: [0.0],
 46699.0: [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0],
 238412.0: [0.0, 0.0, 0.0, 0.0],
 156961.0: [0.0],
 45576.0: [0.0, 0.0, 0.

In [42]:
action_count_last_month = {}
for key in actions_last_month.keys():
    action_count_last_month[key] = len(actions_last_month[key])
action_count_last_month

{379824.0: 6,
 141307.0: 9,
 252255.0: 1,
 309566.0: 1,
 206032.0: 14,
 182119.0: 2,
 68225.0: 1,
 109919.0: 1,
 342023.0: 2,
 287829.0: 1,
 255960.0: 1,
 380420.0: 1,
 176150.0: 2,
 407671.0: 6,
 244552.0: 2,
 46507.0: 3,
 191162.0: 2,
 174308.0: 4,
 134532.0: 1,
 355159.0: 1,
 62942.0: 1,
 353962.0: 6,
 319670.0: 3,
 210578.0: 3,
 101110.0: 4,
 149798.0: 1,
 6018.0: 2,
 281262.0: 1,
 314519.0: 1,
 205257.0: 1,
 117839.0: 1,
 46699.0: 13,
 238412.0: 4,
 156961.0: 1,
 45576.0: 6,
 214766.0: 2,
 5524.0: 1,
 218669.0: 1,
 272518.0: 1,
 277770.0: 1,
 386295.0: 1,
 402251.0: 4,
 254609.0: 4,
 84043.0: 1,
 120820.0: 1,
 291453.0: 1,
 280293.0: 1,
 291386.0: 1,
 313634.0: 4,
 60260.0: 1,
 112997.0: 2,
 305925.0: 2,
 199429.0: 1,
 185381.0: 1,
 144281.0: 2,
 33618.0: 1,
 276857.0: 3,
 130186.0: 4,
 376338.0: 4,
 867.0: 1,
 338395.0: 2,
 335587.0: 6,
 409283.0: 1,
 165355.0: 5,
 55784.0: 1,
 327782.0: 8,
 264657.0: 6,
 52751.0: 2,
 33745.0: 4,
 316235.0: 10,
 380439.0: 2,
 86171.0: 1,
 408759.

In [43]:
total_action_count_last_month = sum(action_count_last_month.values())
total_action_count_last_month

366

In [44]:
action_count_ratio_last_month = {}
for key in action_count_last_month:
    action_count_ratio_last_month[key] = action_count_last_month[key]/total_action_count_last_month
action_count_ratio_last_month

{379824.0: 0.01639344262295082,
 141307.0: 0.02459016393442623,
 252255.0: 0.00273224043715847,
 309566.0: 0.00273224043715847,
 206032.0: 0.03825136612021858,
 182119.0: 0.00546448087431694,
 68225.0: 0.00273224043715847,
 109919.0: 0.00273224043715847,
 342023.0: 0.00546448087431694,
 287829.0: 0.00273224043715847,
 255960.0: 0.00273224043715847,
 380420.0: 0.00273224043715847,
 176150.0: 0.00546448087431694,
 407671.0: 0.01639344262295082,
 244552.0: 0.00546448087431694,
 46507.0: 0.00819672131147541,
 191162.0: 0.00546448087431694,
 174308.0: 0.01092896174863388,
 134532.0: 0.00273224043715847,
 355159.0: 0.00273224043715847,
 62942.0: 0.00273224043715847,
 353962.0: 0.01639344262295082,
 319670.0: 0.00819672131147541,
 210578.0: 0.00819672131147541,
 101110.0: 0.01092896174863388,
 149798.0: 0.00273224043715847,
 6018.0: 0.00546448087431694,
 281262.0: 0.00273224043715847,
 314519.0: 0.00273224043715847,
 205257.0: 0.00273224043715847,
 117839.0: 0.00273224043715847,
 46699.0: 0.0

In [45]:
df_user['last month action count'] = 0
df_user['last month action count ratio'] = 0
for index,row in df_user.iterrows():
        if row['user_id'] in action_count_last_month.keys():
            df_user.at[index, 'last month action count'] = action_count_last_month[row['user_id']]
        if row['user_id'] in action_count_ratio_last_month.keys():
            df_user.at[index, 'last month action count ratio'] = action_count_ratio_last_month[row['user_id']]

df_user

,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio,1111 activity ratio,last week action count,last week action count ratio,last week activity ratio,last month action count,last month action count ratio
0,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393
1,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393
2,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393
3,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393
4,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197
381,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197
382,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197
383,95362,1,0.002597,1,0.005917,1,0.004386,1.0,1,0.002817,1.0,1,0.002732


In [46]:
activity_ratio_last_month = {}
df_user['last month activity ratio'] = 0
for index,row in df_user.iterrows():
    if df_user.at[index,'last month action count'] != 0:
        df_user.at[index, 'last month activity ratio'] = (df_user.at[index,'last month action count']/df_user.at[index,'action count'])
df_user

,user_id,action count,action count ratio,day count,product diversity,1111 action count,1111 action count ratio,1111 activity ratio,last week action count,last week action count ratio,last week activity ratio,last month action count,last month action count ratio,last month activity ratio
0,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393,1.0
1,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393,1.0
2,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393,1.0
3,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393,1.0
4,379824,6,0.015584,2,0.005917,3,0.013158,0.5,6,0.016901,1.0,6,0.016393,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197,1.0
381,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197,1.0
382,122632,3,0.007792,2,0.005917,0,0.000000,0.0,3,0.008451,1.0,3,0.008197,1.0
383,95362,1,0.002597,1,0.005917,1,0.004386,1.0,1,0.002817,1.0,1,0.002732,1.0


In [47]:
print("TOTAL ACTIONS: ", total_action_count)
print("TOTAL ACTIONS LATEST MONTH: ",total_action_count_last_month)
print("TOTAL ACTIONS LATEST WEEK: ",total_action_count_last_week)
print("TOTAL ACTIONS 1111: ",total_action_count_11)

TOTAL ACTIONS:  385
TOTAL ACTIONS LATEST MONTH:  366
TOTAL ACTIONS LATEST WEEK:  355
TOTAL ACTIONS 1111:  228
